In [ ]:

# DB_PATH = 'diagnoses.db'
# cancer = load_breast_cancer()

# AUTHENTICATION = [("admin", "admin123")]

# # Database setup
# @contextmanager
# def get_db_connection():
#     """Create a thread-safe database connection"""
#     conn = sqlite3.connect(DB_PATH)
#     conn.row_factory = sqlite3.Row
#     try:
#         yield conn
#     finally:
#         conn.close()

# def initialize_database():
#     """Create database tables with correct schema"""
#     with get_db_connection() as conn:
#         try:
#             conn.execute("""
#                 CREATE TABLE IF NOT EXISTS diagnoses (
#                     patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
#                     patient_name TEXT NOT NULL,
#                     contact_number TEXT NOT NULL,
#                     address TEXT NOT NULL,
#                     age INTEGER NOT NULL,
#                     features TEXT NOT NULL,
#                     diagnosis TEXT NOT NULL,
#                     confidence REAL NOT NULL,
#                     timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
#                 )""")
#             conn.commit()
#             print("Database initialized")
#         except Exception as e:
#             print(f"Database error: {e}")

# # Model setup
# def load_model():
#     """Load or create ML model"""
#     try:
#         model = joblib.load('breast_cancer_model.pkl')
#         scaler = joblib.load('scaler.pkl')
#     except:
#         X_train, X_test, y_train, y_test = train_test_split(
#             cancer.data, cancer.target, test_size=0.2, random_state=42
#         )
#         scaler = StandardScaler().fit(X_train)
#         model = SVC(probability=True).fit(scaler.transform(X_train), y_train)
#         joblib.dump(model, 'breast_cancer_model.pkl')
#         joblib.dump(scaler, 'scaler.pkl')
#     return model, scaler

# model, scaler = load_model()

# def predict(patient_name, contact_number, address, age, *features):
#     """Handle prediction and storage of diagnosis results"""
#     try:
#         # Convert features to numpy array
#         input_data = np.array(features).reshape(1, -1)
        
#         # Validate input data
#         if np.isnan(input_data).any():
#             return "Invalid/missing medical values detected", pd.DataFrame()

#         # Scale the input data
#         scaled_data = scaler.transform(input_data)

#         # Make prediction
#         prediction = model.predict(scaled_data)[0]
#         probas = model.predict_proba(scaled_data)
        
#         # Get confidence score
#         confidence = probas[0].max() if probas is not None and len(probas) > 0 else 0.0
#         diagnosis = "Malignant" if prediction == 0 else "Benign"
        
#         # Store diagnosis
#         if store_diagnosis(patient_name, contact_number, address, age, 
#                          input_data[0], diagnosis, confidence):
#             # Return formatted result
#             result = f"{diagnosis} ({confidence:.2%} confidence)"
#             return result, view_diagnoses()
        
#         return "Diagnosis failed - storage error", pd.DataFrame()
    
#     except Exception as e:
#         return f"System Error: {str(e)}", pd.DataFrame()

# # Core functions
# def store_diagnosis(patient_name, contact_number, address, age, features, diagnosis, confidence):
#     """Store diagnosis with auto-generated PID"""
#     try:
#         features_dict = {name: float(value) for name, value in zip(cancer.feature_names, features)}
        
#         with get_db_connection() as conn:
#             cursor = conn.cursor()
#             cursor.execute("""
#                 INSERT INTO diagnoses (
#                     patient_name, contact_number, address, age,
#                     features, diagnosis, confidence
#                 ) VALUES (?, ?, ?, ?, ?, ?, ?)
#             """, (
#                 patient_name, contact_number, address, int(age),
#                 json.dumps(features_dict), diagnosis, float(confidence)
#             ))
#             conn.commit()
            
#             pid = f"PID{cursor.lastrowid:03d}"
#             print(f"Stored diagnosis for {pid}")
#             return True
#     except Exception as e:
#         print(f"Storage error: {e}")
#         return False

# def view_diagnoses(limit=5):
#     """Retrieve recent diagnoses with formatted PID"""
#     try:
#         with get_db_connection() as conn:
#             cursor = conn.cursor()
#             cursor.execute(f"""
#                 SELECT 
#                     patient_id,
#                     patient_name,
#                     contact_number,
#                     address,
#                     age,
#                     diagnosis, 
#                     confidence, 
#                     timestamp 
#                 FROM diagnoses 
#                 ORDER BY timestamp DESC 
#                 LIMIT {limit}
#             """)
            
#             rows = cursor.fetchall()
#             if not rows:
#                 return pd.DataFrame()
            
#             df = pd.DataFrame(rows, columns=[
#                 'patient_id', 'patient_name', 'contact_number',
#                 'address', 'age', 'diagnosis', 'confidence', 'timestamp'
#             ])
            
#             # Formatting
#             df['patient_id'] = df['patient_id'].apply(lambda x: f"PID{x:03d}")
#             df['confidence'] = df['confidence'].map(lambda x: f"{x:.2%}")
#             df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%Y-%m-%d %H:%M')
            
#             return df.rename(columns={
#                 'patient_id': 'PID',
#                 'patient_name': 'Name',
#                 'contact_number': 'Contact',
#                 'address': 'Address',
#                 'age': 'Age',
#                 'diagnosis': 'Diagnosis',
#                 'confidence': 'Confidence',
#                 'timestamp': 'Timestamp'
#             })
#     except Exception as e:
#         print(f"Retrieval error: {e}")
#         return pd.DataFrame()

# def generate_report(patient_id):
#     """Generate PDF report for a specific patient"""
#     try:
#         # Extract numeric ID from PID
#         raw_id = patient_id.replace("PID", "").lstrip("0")
#         if not raw_id.isdigit():
#             raise ValueError("Invalid PID format")
        
#         with get_db_connection() as conn:
#             cursor = conn.cursor()
#             cursor.execute("""
#                 SELECT * FROM diagnoses 
#                 WHERE patient_id = ?
#             """, (int(raw_id),))
            
#             record = cursor.fetchone()
            
#             if not record:
#                 print(f"No record found for {patient_id}")
#                 return None

#         # Create PDF with secure temporary file
#         styles = getSampleStyleSheet()
#         with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as tmpfile:
#             filename = tmpfile.name

#         doc = SimpleDocTemplate(filename, pagesize=letter)
#         story = []

#         # Title
#         title = Paragraph("Medical Diagnosis Report", styles["Title"])
#         story.append(title)
#         story.append(Spacer(1, 12))

#         # Patient Information
#         patient_info = [
#             f"<b>Patient ID:</b> {patient_id}",
#             f"<b>Name:</b> {record['patient_name']}",
#             f"<b>Age:</b> {record['age']}",
#             f"<b>Contact:</b> {record['contact_number']}",
#             f"<b>Address:</b> {record['address']}"
#         ]
        
#         for line in patient_info:
#             p = Paragraph(line, styles["Normal"])
#             story.append(p)

#         story.append(Spacer(1, 24))

#         # Diagnosis Details
#         diagnosis_content = [
#             f"<b>Diagnosis:</b> {record['diagnosis']}",
#             f"<b>Confidence:</b> {float(record['confidence']):.2%}",
#             f"<b>Date:</b> {record['timestamp']}"
#         ]
        
#         for line in diagnosis_content:
#             p = Paragraph(line, styles["Normal"])
#             story.append(p)

#         # Features Table
#         story.append(Spacer(1, 24))
#         features = json.loads(record['features'])
#         feature_data = [[k, f"{v:.4f}"] for k, v in features.items()]
#         feature_table = Table([["Measurement", "Value"]] + feature_data)
#         feature_table.setStyle(TableStyle([
#             ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
#             ('TEXTCOLOR', (0,0), (-1,0), colors.black),
#             ('ALIGN', (0,0), (-1,-1), 'CENTER'),
#             ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
#             ('FONTSIZE', (0,0), (-1,0), 10),
#             ('BOTTOMPADDING', (0,0), (-1,0), 12),
#             ('BACKGROUND', (0,1), (-1,-1), colors.white),
#             ('GRID', (0,0), (-1,-1), 1, colors.black)
#         ]))
#         story.append(feature_table)

#         doc.build(story)
#         return filename
        
#     except Exception as e:
#         print(f"Report generation error: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         return None

# def generate_statistical_report():
#     """Generate PDF report with system statistics"""
#     try:
#         # Create PDF with secure temporary file
#         with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as tmpfile:
#             filename = tmpfile.name

#         doc = SimpleDocTemplate(filename, pagesize=letter)
#         story = []
#         styles = getSampleStyleSheet()

#         # Title
#         title = Paragraph("System Statistical Report", styles["Title"])
#         story.append(title)
#         story.append(Spacer(1, 24))

#         # Database queries
#         with get_db_connection() as conn:
#             cursor = conn.cursor()
            
#             # Total patients
#             cursor.execute("SELECT COUNT(*) FROM diagnoses")
#             total_patients = cursor.fetchone()[0]
            
#             # Diagnosis distribution
#             cursor.execute("SELECT diagnosis, COUNT(*) FROM diagnoses GROUP BY diagnosis")
#             diagnosis_data = cursor.fetchall()
            
#             # Date range
#             cursor.execute("SELECT MIN(timestamp), MAX(timestamp) FROM diagnoses")
#             date_range = cursor.fetchone()

#         # Summary section
#         summary = [
#             f"<b>Total Patients:</b> {total_patients}",
#             f"<b>Date Range:</b> {date_range[0]} to {date_range[1]}"
#         ]
#         for line in summary:
#             story.append(Paragraph(line, styles["Normal"]))
        
#         story.append(Spacer(1, 24))

#         # Diagnosis distribution table
#         if diagnosis_data:
#             total = sum(count for _, count in diagnosis_data)
#             table_data = [["Diagnosis", "Count", "Percentage"]]
#             for diag, count in diagnosis_data:
#                 pct = (count / total) * 100 if total > 0 else 0
#                 table_data.append([diag, str(count), f"{pct:.1f}%"])
            
#             diag_table = Table(table_data)
#             diag_table.setStyle(TableStyle([
#                 ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
#                 ('TEXTCOLOR', (0,0), (-1,0), colors.black),
#                 ('ALIGN', (0,0), (-1,-1), 'CENTER'),
#                 ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
#                 ('GRID', (0,0), (-1,-1), 1, colors.black)
#             ]))
#             story.append(diag_table)

#         doc.build(story)
#         return filename
        
#     except Exception as e:
#         print(f"Statistical report error: {str(e)}")
#         return None

# def generate_patient_list_report():
#     """Generate PDF report of all patients"""
#     try:
#         with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as tmpfile:
#             filename = tmpfile.name

#         doc = SimpleDocTemplate(filename, pagesize=landscape(letter))
#         story = []
#         styles = getSampleStyleSheet()

#         # Title
#         title = Paragraph("Complete Patient List", styles["Title"])
#         story.append(title)
#         story.append(Spacer(1, 24))

#         # Table data
#         with get_db_connection() as conn:
#             cursor = conn.cursor()
#             cursor.execute("""
#                 SELECT patient_id, patient_name, contact_number, 
#                        address, diagnosis, confidence, timestamp 
#                 FROM diagnoses
#             """)
#             rows = cursor.fetchall()

#         table_data = [["PID", "Name", "Contact", "Address", "Diagnosis", "Confidence", "Date"]]
#         for row in rows:
#             table_data.append([
#                 f"PID{row['patient_id']:03d}",
#                 row['patient_name'],
#                 row['contact_number'],
#                 row['address'],
#                 row['diagnosis'],
#                 f"{float(row['confidence']):.2%}",
#                 row['timestamp'][:10]  # Truncate timestamp to date
#             ])

#         # Create table
#         table = Table(table_data, repeatRows=1)
#         table.setStyle(TableStyle([
#             ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
#             ('TEXTCOLOR', (0,0), (-1,0), colors.black),
#             ('ALIGN', (0,0), (-1,-1), 'LEFT'),
#             ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
#             ('FONTSIZE', (0,0), (-1,-1), 8),
#             ('GRID', (0,0), (-1,-1), 1, colors.black),
#             ('WORDWRAP', (3,1), (3,-1), True)  # Wrap address column
#         ]))
#         story.append(table)

#         doc.build(story)
#         return filename
        
#     except Exception as e:
#         print(f"Patient list error: {str(e)}")
#         return None

# def generate_patient_list_report():
#     """Generate PDF report of all patients with contact info and diagnoses"""
#     try:
#         with get_db_connection() as conn:
#             cursor = conn.cursor()
#             cursor.execute("""
#                 SELECT patient_id, patient_name, contact_number, address,
#                        diagnosis, confidence, timestamp 
#                 FROM diagnoses 
#                 ORDER BY timestamp DESC
#             """)
#             rows = cursor.fetchall()

#             if not rows:
#                 return None

#             # Create PDF
#             styles = getSampleStyleSheet()
#             filename = tempfile.mktemp(suffix=".pdf")
#             doc = SimpleDocTemplate(filename, pagesize=letter)
#             story = []

#             # Title
#             title = Paragraph("<para align=center><b>Complete Patient List Report</b></para>", styles["Title"])
#             story.append(title)
#             story.append(Spacer(1, 24))

#             # Table data
#             table_data = [
#                 ["PID", "Name", "Phone", "Address", "Diagnosis", "Confidence", "Date"]
#             ]
            
#             for row in rows:
#                 table_data.append([
#                     f"PID{row['patient_id']:03d}",
#                     row['patient_name'],
#                     row['contact_number'],
#                     row['address'],
#                     row['diagnosis'],
#                     f"{float(row['confidence']):.2%}",
#                     pd.to_datetime(row['timestamp']).strftime('%Y-%m-%d')
#                 ])

#             # Create table
#             t = Table(table_data)
#             t.setStyle([
#                 ('BACKGROUND', (0,0), (-1,0), '#CCCCCC'),
#                 ('TEXTCOLOR', (0,0), (-1,0), '#000000'),
#                 ('ALIGN', (0,0), (-1,-1), 'CENTER'),
#                 ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
#                 ('FONTSIZE', (0,0), (-1,0), 8),
#                 ('BOTTOMPADDING', (0,0), (-1,0), 12),
#                 ('BACKGROUND', (0,1), (-1,-1), '#FFFFFF'),
#                 ('GRID', (0,0), (-1,-1), 1, '#000000'),
#                 ('FONTSIZE', (0,1), (-1,-1), 8),
#                 ('WORDWRAP', (2,1), (3,-1), True)  # Wrap address and phone
#             ])
            
#             story.append(t)
#             doc.build(story)
#             return filename
            
#     except Exception as e:
#         print(f"Patient list report error: {e}")
#         return None

# def create_history_interface():
#     with gr.Blocks(title="Diagnosis History") as interface:
#         gr.Markdown("# Historical Diagnosis Records")
        
#         # Search and Refresh Section
#         with gr.Row():
#             search_input = gr.Textbox(
#                 label="Search by Patient ID or Name",
#                 placeholder="Enter PID (e.g., PID001) or name...",
#                 value=""
#             )
#             limit_selector = gr.Dropdown(
#                 choices=[5, 10, 25, 50, 100], 
#                 value=10, 
#                 label="Max Records to Show"
#             )
#             refresh_btn = gr.Button("🔃 Refresh", variant="secondary")
        
#         # Main Records Table
#         records_table = gr.DataFrame(
#             headers=["PID", "Name", "Contact", "Address", "Age", "Diagnosis", "Confidence", "Timestamp"],
#             interactive=False,
#             label="Diagnosis History"
#         )

#         # Report Generation Section
#         with gr.Row():
#             with gr.Column(scale=2):
#                 report_type = gr.Dropdown(
#                     choices=["Individual Patient Report", "System Statistics Report", "All Patients List"],
#                     value="Individual Patient Report",
#                     label="Select Report Type"
#                 )
#                 report_pid = gr.Textbox(
#                     label="Enter PID for Individual Report",
#                     visible=True,
#                     placeholder="PID000"
#                 )
            
#             with gr.Column(scale=1):
#                 generate_btn = gr.Button("📄 Generate Report", variant="primary")
#                 report_download = gr.File(label="Download Report", visible=False)
        
#         # Statistical Report Controls
#         with gr.Row():
#             stats_refresh_btn = gr.Button("🔄 Update Statistics", variant="secondary")
#             stats_download = gr.File(label="Statistics Report", visible=False)

#         def toggle_pid_input(report_type):
#             return gr.update(visible=report_type == "Individual Patient Report")

#         def search_history(search_term, limit):
#             try:
#                 with get_db_connection() as conn:
#                     cursor = conn.cursor()
#                     limit = int(limit)
                    
#                     if not search_term.strip():
#                         # Get all records when search is empty
#                         cursor.execute("""
#                             SELECT patient_id, patient_name, contact_number,
#                                    address, age, diagnosis, confidence, timestamp
#                             FROM diagnoses 
#                             ORDER BY timestamp DESC 
#                             LIMIT ?
#                         """, (limit,))
#                     else:
#                         # Handle PID search
#                         if search_term.upper().startswith("PID"):
#                             try:
#                                 pid = int(search_term[3:].lstrip('0') or 0)
#                                 cursor.execute("""
#                                     SELECT patient_id, patient_name, contact_number,
#                                            address, age, diagnosis, confidence, timestamp
#                                     FROM diagnoses 
#                                     WHERE patient_id = ?
#                                     ORDER BY timestamp DESC 
#                                     LIMIT ?
#                                 """, (pid, limit))
#                             except ValueError:
#                                 return pd.DataFrame()
#                         else:
#                             # Handle name search
#                             cursor.execute("""
#                                 SELECT patient_id, patient_name, contact_number,
#                                        address, age, diagnosis, confidence, timestamp
#                                 FROM diagnoses 
#                                 WHERE patient_name LIKE ?
#                                 ORDER BY timestamp DESC 
#                                 LIMIT ?
#                             """, (f"%{search_term}%", limit))

#                     rows = cursor.fetchall()
                    
#                     # Format results
#                     if rows:
#                         df = pd.DataFrame(rows, columns=[
#                             'patient_id', 'patient_name', 'contact_number',
#                             'address', 'age', 'diagnosis', 'confidence', 'timestamp'
#                         ])
                        
#                         # Apply formatting
#                         df['patient_id'] = df['patient_id'].apply(lambda x: f"PID{x:03d}")
#                         df['confidence'] = df['confidence'].apply(lambda x: f"{float(x):.2%}")
#                         df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%Y-%m-%d %H:%M')
                        
#                         return df.rename(columns={
#                             'patient_id': 'PID',
#                             'patient_name': 'Name',
#                             'contact_number': 'Contact',
#                             'address': 'Address',
#                             'age': 'Age',
#                             'diagnosis': 'Diagnosis',
#                             'confidence': 'Confidence',
#                             'timestamp': 'Timestamp'
#                         })
#                     return pd.DataFrame()
                    
#             except Exception as e:
#                 print(f"Search error: {e}")
#                 return pd.DataFrame()

#         def handle_report_generation(report_type, pid):
#             try:
#                 if report_type == "Individual Patient Report":
#                     if not pid.startswith("PID"):
#                         return None, gr.update(visible=False)
#                     filename = generate_report(pid)
#                 elif report_type == "All Patients List":
#                     filename = generate_patient_list_report()
#                 else:
#                     filename = generate_statistical_report()
                
#                 if filename:
#                     return filename, gr.update(visible=True, value=filename)
#                 return None, gr.update(visible=False)
#             except Exception as e:
#                 print(f"Report generation error: {e}")
#                 return None, gr.update(visible=False)

#         # Event handlers
#         report_type.change(
#             fn=toggle_pid_input,
#             inputs=report_type,
#             outputs=report_pid
#         )
        
#         refresh_btn.click(
#             fn=search_history,
#             inputs=[search_input, limit_selector],
#             outputs=records_table
#         )
        
#         generate_btn.click(
#             fn=handle_report_generation,
#             inputs=[report_type, report_pid],
#             outputs=[report_download, report_download]
#         )
        
#         stats_refresh_btn.click(
#             fn=lambda: (generate_statistical_report(), gr.update(visible=True)),
#             outputs=[stats_download, stats_download]
#         )
        
#         # Initial load
#         interface.load(
#             fn=lambda: search_history("", 10),
#             outputs=records_table
#         )
    
#     return interface



# def main():
#     initialize_database()
    
#     # Create interfaces
#     prediction_interface = gr.Interface(
#         fn=predict,
#         inputs=[
#             gr.Textbox(label="Patient Name", placeholder="Full name..."),
#             gr.Textbox(label="Contact Number", placeholder="Phone number..."),
#             gr.Textbox(label="Address", placeholder="Current address..."),
#             gr.Number(label="Age", minimum=0, maximum=120),
#             *[gr.Number(label=name) for name in cancer.feature_names]
#         ],
#         outputs=[
#             gr.Textbox(label="Diagnosis Result"),
#             gr.DataFrame(label="Recent Diagnoses")
#         ],
#         title="Breast Cancer Diagnosis",
#         description="Submit new patient data for diagnosis",
#         allow_flagging="never"
#     )
    
#     history_interface = create_history_interface()
    
#     # Create tabbed interface
#     app = gr.TabbedInterface(
#         [prediction_interface, history_interface],
#         ["New Diagnosis", "View History"],
#         title="Breast Cancer Classification System"
#     )
    
#     # Corrected launch parameters
#     app.launch(
#         share=True,
#         auth=AUTHENTICATION,
#         auth_message="Please login with admin credentials"
#     )

# if __name__ == "__main__":
#     main()